## PROJECT TITLE: EXPLORATORY DATA ANALYSIS

|ID   |Name   |Resposible for|% contribution|
|---|---|---|---|
|20127094   |Nguyễn Minh Văn|   |   |
|20127247   |Phan Xuân Nam   |Data Pre-processing   |100%   |
|20127333   |Nguyễn Duy Thịnh   |Data Crawling   |100%   |
|20127380   |Dương Minh Tùng   |   |   |
|20127406   |Thái Cẩm Phong   |Data Crawling   |100%   |

### Step 1: Collecting data

### Step 2: Pre-processing data

### Step 2.1: Importing important libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Step 2.2: Importing datasets
There are 3 files: "playlists.csv", "tracks.csv", "users.csv"

In [2]:
playlists_df = pd.read_csv("playlists.csv")
tracks_df = pd.read_csv("tracks.csv")
users_df = pd.read_csv("users.csv")

In [3]:
print(playlists_df.keys())
print(tracks_df.keys())
print(users_df.keys())

Index(['Unnamed: 0', 'id', 'title', 'author', 'tracks', 'likes', 'reposts',
       'release'],
      dtype='object')
Index(['Unnamed: 0', 'id', 'title', 'author', 'plays', 'likes', 'reposts',
       'release'],
      dtype='object')
Index(['Unnamed: 0', 'id', 'name', 'followers', 'following', 'tracks'], dtype='object')


Hmmmm, there is an extra column called "Unnamed: 0". Let's remove that for all 3 dataframes

In [4]:
tracks_df.drop(columns='Unnamed: 0', inplace=True)
users_df.drop(columns='Unnamed: 0', inplace=True)
playlists_df.drop(columns='Unnamed: 0', inplace=True)

In [5]:
users_df.head()

,id,name,followers,following,tracks
0,1,TMN Playlisted,21.6K,116,2
1,2,nymano,69.1K,512,79
2,3,Agami,14.7K,598,1
3,4,Levi Patel,14.7K,433,19
4,5,Alice Baldwin,"4,680",117,18


Seems good, let the pre-processing begin!

### Step 2.3: Pre-processing
These are methods that we will use for data pre-processing of each dataframe:
- Check for meaning of each column
- Check the number of columns and rows 
- Check for duplicated rows
- Check for missing data
- Check for the min/max values, mean 
- Check for the data type of each column and modify values (if needed)

Let's start with the "users" dataframe. We check for the first 5 rows of it. 

In [6]:
users_df.head()

,id,name,followers,following,tracks
0,1,TMN Playlisted,21.6K,116,2
1,2,nymano,69.1K,512,79
2,3,Agami,14.7K,598,1
3,4,Levi Patel,14.7K,433,19
4,5,Alice Baldwin,"4,680",117,18


The meaning of each column is:
+ id: the id of the user (categorical)
+ name: the username of the user (categorical)
+ followers: number of followers (numeric)
+ following: number of people that a user follows (numeric)
+ tracks: the number of tracks (numeric)

The name "tracks" is quite ambiguous, let's rename it so it's clearer to understand

In [7]:
users_df.rename(columns={"tracks": "NumTracks"}, inplace=True)

We check for any duplicated rows in the dataframe

In [8]:
check_user_dup = users_df.duplicated().any()
if check_user_dup:
    print("There are duplicated rows")
else:
    print("There is no duplicated rows")

There are duplicated rows


Wow, there are duplicated rows. We have to make each row unique and remove duplicated ones

In [9]:
users_df.drop_duplicates(inplace=True)

After removing duplicated values, we check for the number of rows and columns

In [10]:
print("Number of rows: "+str(users_df.shape[0]))
print("Number of columns: "+str(users_df.shape[1]))

Number of rows: 2895
Number of columns: 5


We check if there are any missing data in the dataframe

In [11]:
col_users_key = list(users_df.keys())
percent_missing = users_df[col_users_key].isnull().sum() * 100 / len(users_df)
percent_missing

id          0.00000
name        0.13817
followers   0.24180
following   0.27634
NumTracks   0.27634
dtype: float64

Since there are very small number of rows that exist "null" values, we can drop them 

In [12]:
users_df.dropna(inplace=True)
print("Number of rows: "+str(users_df.shape[0]))
print("Number of columns: "+str(users_df.shape[1]))

Number of rows: 2886
Number of columns: 5


Next, we see what the data type of each column

In [13]:
users_df.dtypes

id            int64
name         object
followers    object
following    object
NumTracks    object
dtype: object

We find that the number of followers, following and num tracks are not in the correct type, it should be in "int" type

In [14]:
def change_number(s):
    to_change = list(set(s))
    change = []
    for item in to_change:
        temp = item
        if ("K" in item) and ("." not in item): 
            temp = item.replace("K", "000")
        elif ("K" in item) and ("." in item): 
            temp = item.replace("K", "00").replace(".", "")
        elif ("M" in item) and ("." not in item): 
            temp = item.replace("M", "000000")
        elif ("M" in item) and ("." in item) and (len(temp)==4): 
            temp = item.replace("M", "00000").replace(".", "")
        elif ("M" in item) and ("." in item) and (len(temp)==5): 
            temp = item.replace("M", "0000").replace(".", "")
        elif ("," in item):
            temp = item.replace(",", "")
        change.append(temp)
    return to_change, change

In [15]:
# users_df['followers'] = users_df['followers'].str.replace(',','')
# user_to_change, user_change = change_number(users_df['followers'])
# users_df['NumTracks'] = users_df['NumTracks'].replace(['12.1K'], '12100')
# users_df['NumTracks'] = users_df['NumTracks'].str.replace(',','')
# users_df["NumTracks"] = users_df['NumTracks'].astype(int)
# users_df['followers'] = users_df['followers'].replace(user_to_change, user_change)
# users_df['followers'] = users_df['followers'].astype(int)
# users_df['following'] = users_df['following'].str.replace(',','')
# users_df['following'] = users_df['following'].astype(int)

In [16]:
num_track_to_change, num_track_change = change_number(users_df['NumTracks'])
users_df['NumTracks'] = users_df['NumTracks'].replace(num_track_to_change, num_track_change)
follower_to_change, follower_change = change_number(users_df['followers'])
users_df['followers'] = users_df['followers'].replace(follower_to_change, follower_change)
following_to_change, following_change = change_number(users_df['following'])
users_df['following'] = users_df['following'].replace(following_to_change, following_change)

users_df["NumTracks"] = users_df['NumTracks'].astype(int)
users_df['followers'] = users_df['followers'].astype(int)
users_df['following'] = users_df['following'].astype(int)

Let's have a look at how the data distribute

In [17]:
users_df.describe().drop(columns='id')

,followers,following,NumTracks
count,2886.00000,2886.00000,2886.00000
mean,178472.59459,130.61296,97.39744
std,810164.81229,310.78059,444.55766
min,0.00000,0.00000,0.00000
25%,174.50000,1.00000,0.00000
50%,2549.00000,15.00000,13.00000
75%,13300.00000,94.75000,50.00000
max,9510000.00000,2027.00000,12100.00000


Moving on, we work with the "tracks". We look for the first 5 rows

In [18]:
tracks_df.head()

,id,title,author,plays,likes,reposts,release
0,1,solitude,2,"18,184,198","298,037","14,517",7 years ago\n7 years ago
1,2,quand la pluie tombe (also uploaded for The Vi...,2,"11,928,581","155,823","7,556",8 years ago\n8 years ago
2,3,As she passes,4,"15,385,582","228,383","7,815",7 years ago\n7 years ago
3,4,Was am Ende,5,"3,865,947","37,123","1,447",3 years ago\n3 years ago
4,5,Making All Things New (Waterman/Espe) - Record...,7,"5,974,821","123,856","3,261",8 years ago\n8 years ago


The meaning of each column is:
+ id: the id of the track (categorical)
+ title: name of the track (categorical)
+ author: person who makes the track (numeric)
+ plays: total play count of a track
+ likes: number of people like a track (numeric)
+ reposts: number of reposting (numeric)
+ release: total years/months/days since the track first released (categorical)

We will change the columns' name to make it easier

In [19]:
tracks_df.rename(columns={"author": "author_id", "plays": "total_plays", "release": "last_release"}, inplace=True)

We check for duplicates in the above dataframe

In [20]:
check_track_dup = tracks_df.duplicated().any()
if check_track_dup:
    print("There are duplicated rows")
else:
    print("There is no duplicated rows")

There is no duplicated rows


Awesome! Now we look for the number of rows and columns

In [21]:
print("Number of rows: "+str(tracks_df.shape[0]))
print("Number of columns: "+str(tracks_df.shape[1]))

Number of rows: 2074
Number of columns: 7


Okay, let's look for missing values, shall we?

In [22]:
col_tracks_key = list(tracks_df.keys())
percent_missing = tracks_df[col_tracks_key].isnull().sum() * 100 / len(tracks_df)
percent_missing

id             0.00000
title          0.04822
author_id      0.00000
total_plays    0.00000
likes          0.00000
reposts        0.00000
last_release   0.00000
dtype: float64

Hmmm, there are some missing titles. Since the missing ratio is too insignificant, we can dop them.

In [23]:
tracks_df.dropna(inplace=True)
print("Number of rows: "+str(tracks_df.shape[0]))
print("Number of columns: "+str(tracks_df.shape[1]))

Number of rows: 2073
Number of columns: 7


Now, the important part is to check for the data type and convert them accordingly to its supposingly correct one

In [24]:
tracks_df.dtypes

id               int64
title           object
author_id        int64
total_plays     object
likes           object
reposts         object
last_release    object
dtype: object

In [25]:
total_play_to_change, total_play_change = change_number(tracks_df['total_plays'])
tracks_df['total_plays'] = tracks_df['total_plays'].replace(total_play_to_change, total_play_change)
tracks_df['total_plays'] = tracks_df['total_plays'].astype(int)
likes_to_change, likes_change = change_number(tracks_df['likes'])
tracks_df['likes'] = tracks_df['likes'].replace(likes_to_change, likes_change)
tracks_df['likes'] = tracks_df['likes'].astype(int)
reposts_to_change, reposts_change = change_number(tracks_df['reposts'])
tracks_df['reposts'] = tracks_df['reposts'].replace(reposts_to_change, reposts_change)
tracks_df['reposts'] = tracks_df['reposts'].astype(int)

release_to_change = list(set(tracks_df['last_release']))
release_change = [x.split("\n")[1] for x in release_to_change] 
tracks_df['last_release'] = tracks_df['last_release'].replace(release_to_change, release_change)
tracks_df.head()

,id,title,author_id,total_plays,likes,reposts,last_release
0,1,solitude,2,18184198,298037,14517,7 years ago
1,2,quand la pluie tombe (also uploaded for The Vi...,2,11928581,155823,7556,8 years ago
2,3,As she passes,4,15385582,228383,7815,7 years ago
3,4,Was am Ende,5,3865947,37123,1447,3 years ago
4,5,Making All Things New (Waterman/Espe) - Record...,7,5974821,123856,3261,8 years ago


Let's have a look at how the data distribute

In [26]:
tracks_df.describe().drop(columns=['id', 'author_id'])

,total_plays,likes,reposts
count,2073.00000,2073.00000,2073.00000
mean,10914330.15388,149387.32465,9689.08056
std,30547982.35685,359681.87191,38807.33667
min,0.00000,0.00000,0.00000
25%,128689.00000,1642.00000,87.00000
50%,1001579.00000,15059.00000,763.00000
75%,6712317.00000,106360.00000,5377.00000
max,326472925.00000,3164571.00000,1402199.00000


Finally, we will work with the "playlists"

In [27]:
playlists_df.head()

,id,title,author,tracks,likes,reposts,release
0,1,Acid Jazz,1,"1,2",387K,35.8K,6 years ago
1,2,Ambient piano,3,"3,4",149K,"9,611",6 years ago
2,3,As Beautiful As It Sounds,6,"5,6",80.1K,"4,509",7 years ago
3,4,ahmed,9,"7,8",2,0,2 years ago
4,5,Artists to Watch,12,"9,10",34.2K,"2,089",5 years ago


The meaning of each column is:
+ id: the ID for the playlist
+ title: name of the playlist
+ author: the ID of the author who creates the playlist
+ tracks: the ID of the first 2 tracks of the playlist
+ likes: number of likes
+ reposts: number of reposts
+ release: the years/months/days released

We should change the name of some of the columns for clarity

In [28]:
playlists_df.rename(columns={"author": "author_id", "tracks": "first_2_tracks_id", \
                             "release": "last_release"}, inplace=True)

We check for duplicates in the above dataframe

In [29]:
check_playlist_dup = playlists_df.duplicated().any()
if check_playlist_dup:
    print("There are duplicated rows")
else:
    print("There is no duplicated rows")

There is no duplicated rows


Awesome! Now we look for the number of rows and columns

In [30]:
print("Number of rows: "+str(playlists_df.shape[0]))
print("Number of columns: "+str(playlists_df.shape[1]))

Number of rows: 1045
Number of columns: 7


Okay, let's look for missing values, shall we?

In [31]:
col_playlists_key = list(playlists_df.keys())
percent_missing = playlists_df[col_playlists_key].isnull().sum() * 100 / len(playlists_df)
percent_missing

id                  0.00000
title               0.00000
author_id           0.00000
first_2_tracks_id   0.00000
likes               0.00000
reposts             0.00000
last_release        0.00000
dtype: float64

Awesome, we don't have to drop any rows. Now we move to check the data types and convert them if needed

In [32]:
playlists_df.dtypes

id                    int64
title                object
author_id             int64
first_2_tracks_id    object
likes                object
reposts              object
last_release         object
dtype: object

In [33]:
like_to_change, like_change = change_number(playlists_df['likes'])
repost_to_change, repost_change = change_number(playlists_df['reposts'])

playlists_df['likes'] = playlists_df['likes'].replace(like_to_change, like_change)
playlists_df['likes'] = playlists_df['likes'].astype(int)
playlists_df['reposts'] = playlists_df['reposts'].replace(repost_to_change, repost_change)
playlists_df['reposts'] = playlists_df['reposts'].astype(int)
playlists_df.head()

,id,title,author_id,first_2_tracks_id,likes,reposts,last_release
0,1,Acid Jazz,1,"1,2",387000,35800,6 years ago
1,2,Ambient piano,3,"3,4",149000,9611,6 years ago
2,3,As Beautiful As It Sounds,6,"5,6",80100,4509,7 years ago
3,4,ahmed,9,"7,8",2,0,2 years ago
4,5,Artists to Watch,12,"9,10",34200,2089,5 years ago


Let's have a look at how the data distribute

In [34]:
playlists_df.describe().drop(columns=['id', 'author_id'])

,likes,reposts
count,1045.00000,1045.00000
mean,12334.79904,928.71388
std,37172.86578,2830.86907
min,0.00000,0.00000
25%,3.00000,0.00000
50%,47.00000,4.00000
75%,6926.00000,557.00000
max,515000.00000,43200.00000
